In [ ]:
from pymongo import MongoClient
import pandas as pd

client = MongoClient('localhost', 27017)

db = client['NBA-Stats']

games_cltn = db['Games']
player_stats_cltn = db['Player Stats']
player_cltn = db['players']
player_game_cltn = db['player_game_stats']

In [3]:
from nba_api.stats.endpoints import teamgamelog
teamgamelog.TeamGameLog(team_id=0).get_dict()

{'resource': 'teamgamelog',
 'parameters': {'TeamID': 0,
  'LeagueID': None,
  'Season': '2023-24',
  'SeasonType': 'Regular Season',
  'DateFrom': None,
  'DateTo': None},
 'resultSets': [{'name': 'TeamGameLog',
   'headers': ['Team_ID',
    'Game_ID',
    'GAME_DATE',
    'MATCHUP',
    'WL',
    'W',
    'L',
    'W_PCT',
    'MIN',
    'FGM',
    'FGA',
    'FG_PCT',
    'FG3M',
    'FG3A',
    'FG3_PCT',
    'FTM',
    'FTA',
    'FT_PCT',
    'OREB',
    'DREB',
    'REB',
    'AST',
    'STL',
    'BLK',
    'TOV',
    'PF',
    'PTS'],
   'rowSet': []}]}

In [ ]:
from nba_api.live.nba.endpoints import scoreboard
from nba_api.stats.endpoints import playercareerstats
from nba_api.stats.endpoints import commonallplayers
from nba_api.stats.endpoints import cumestatsplayergames

games = scoreboard.ScoreBoard()
record = games.get_dict()

all_players = commonallplayers.CommonAllPlayers().get_dict()
#player_cltn.insert_one(all_players)
#stats_cltn.insert_one(record)

In [ ]:
from nba_api.stats.endpoints import cumestatsplayergames
query = cumestatsplayergames.CumeStatsPlayerGames(2544, season='2022-23').get_dict()
query
# x = 0
# for i in query['resultSets'][0]['rowSet']:
#     print(x)
#     x += 1

In [ ]:
from nba_api.stats.endpoints import teamgamelogs
#pd.json_normalize(json.loads(teamgamelogs.TeamGameLogs().get_json()))
query = teamgamelogs.TeamGameLogs(season_nullable='2023-24').get_dict()
x = 0
for i in query['resultSets'][0]['rowSet']:
    print(i)
    x += 1

# headers = query['resultSets'][0]['headers']
# results = query['resultSets'][0]['rowSet'][0]

# res = {}
# for key in headers:
#     # res[key] = results[vali]
#     for value in results:
#         res[key] = value
#         results.remove(value)
#         break
# res

In [ ]:

# df = pd.DataFrame(data={"player_id": [0], "player_name": ["test"]}, columns=["player_id", "player_name"])
# i = 0
# while i < 4905:

#     # Define the aggregation pipeline
#     pipeline = [
#         {"$unwind": "$resultSets"},
#         {"$project": {"firstElementInRowSet": {"$arrayElemAt": ["$resultSets.rowSet", i]}}},
#         {"$limit": 1}
#     ]

#     # Execute the aggregation query
#     result = list(player_cltn.aggregate(pipeline))

#     # Access the first element in the "rowSet" array
#     if result:
#         first_element = result[0]["firstElementInRowSet"]
#         print(f"{first_element[0]}, {first_element[2]}")
#         df.loc[i] = [first_element[0], first_element[2]]
#     else:
#         print("No results found.")
#     i += 1
# df

In [ ]:
# ids = pd.Series(df['player_id'])
# names = pd.Series(df['player_name'])
# i = 0
# while i < 4905:
#     player = playercareerstats.PlayerCareerStats(player_id=ids[i])
#     player = player.get_dict()
#     player['player name'] = names[i]
#     player_cltn.insert_one(player)
#     i += 1

In [ ]:
from tqdm.notebook import tqdm


headers = query['resultSets'][0]['headers']
query = query['resultSets'][0]['rowSet']
i = 0
for game in tqdm(query, ):
    results = query[i]
    res = {}
    for key in headers:
        # res[key] = results[vali]
        for value in results:
            res[key] = value
            results.remove(value)
            break
    games_cltn.insert_one(res)
    i += 1
    

In [ ]:
import psycopg2
from warnings import filterwarnings
filterwarnings('ignore')
conn = psycopg2.connect(database="nba_stats",user="postgres",password="DataNerd2023!!",host="localhost")
ids = pd.read_sql("SELECT DISTINCT player_id, name, SUM(games_played) AS total_games FROM player_stats GROUP BY player_id, name ORDER BY total_games DESC", con=conn)
ids = list(pd.Series(ids['player_id']))
ids

In [ ]:
from nba_api.stats.endpoints import playergamelogs
from tqdm.notebook import tqdm
# "2023-24", "2022-23", "2021-22", "2020-21", "2019-20", "2018-19", "2017-18", 
seasons = ["2014-15", "2013-14", "2012-13", "2011-12", "2010-11", "2009-10", "2008-09", "2007-08", "2006-07", "2004-05", "2003-04"]
for season in seasons:
    query = playergamelogs.PlayerGameLogs(season_nullable=season).get_dict()

    headers = query['resultSets'][0]['headers']
    query = query['resultSets'][0]['rowSet']
    i = 0
    for game in tqdm(query, ):
        results = query[i]
        res = {}
        for key in headers:
            for value in results:
                res[key] = value
                results.remove(value)
                break
        player_game_cltn.insert_one(res)
        i += 1
    print(season)

In [ ]:

for i in tqdm(query['resultSets'][0]['rowSet']):
    print(i)
    res = {}
    for key in headers:
            # res[key] = results[vali]
            for value in values:
                res[key] = value
                values.remove(value)
                break
    results = pd.DataFrame(res, index=[0])
    results.to_csv('player_game_stats.csv', index=False, header=False, mode='a')

In [ ]:

from tqdm.notebook import tqdm
query = playergamelogs.PlayerGameLogs().get_dict()
headers = query['resultSets'][0]['headers']
index = 0
for result in tqdm(query):
    values = query['resultSets'][0]['rowSet'][index]
    res = {}
    for key in headers:
            # res[key] = results[vali]
            for value in values:
                res[key] = value
                values.remove(value)
                break
    results = pd.DataFrame(res, index=[0])
    final = pd.concat([df, results])
    index += 1

In [ ]:
import psycopg2
from warnings import filterwarnings
filterwarnings('ignore')
conn = psycopg2.connect(database="nba_stats",user="postgres",password="DataNerd2023!!",host="localhost")
pd.read_sql("SELECT DISTINCT player_id FROM player_stats", con=conn)

In [ ]:
final